In [1]:
import torch
import torchvision
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import time

torch.manual_seed(7)
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('./logs')

In [3]:
random_affine = torchvision.transforms.RandomAffine(degrees=10,
                                                        scale=(0.9, 1.1),
                                                        translate=(0.1, 0.1),
                                                        interpolation=torchvision.transforms.InterpolationMode.BICUBIC)

In [4]:
#定义超参数
BATCH_SIZE = 10

transform1 = transforms.Compose([torchvision.transforms.Resize((700, 1200)),
                                torchvision.transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                #transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))
                                ])

transform2 = transforms.Compose([torchvision.transforms.Resize((700, 1200)),
                                transforms.ToTensor(),
                                #transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))
                                ])

trainset = torchvision.datasets.FGVCAircraft(root='./data',split = "train",
                                        download=True, transform=transform1)
trainloader = torch.utils.data.DataLoader(trainset,batch_size = BATCH_SIZE,
                                          shuffle = True, num_workers=6,pin_memory = True)

testset = torchvision.datasets.FGVCAircraft(root='./data',split = "val",
                                        download=True, transform=transform2)
testloader = torch.utils.data.DataLoader(testset,batch_size = BATCH_SIZE,
                                          shuffle = False, num_workers=6,pin_memory = True)


In [5]:
mynet = torchvision.models.quantization.resnet50(weights=None, progress=True, quantize=False,num_classes=100)
mynet = mynet.train().to(DEVICE)
optimizer = torch.optim.SGD(mynet.parameters(), lr=0.01, momentum=0.9)
loss_func = torch.nn.CrossEntropyLoss()

In [6]:
start = time.process_time()

In [7]:
print(start)

4.421875


In [8]:
lost = 0
mynet = mynet.train().to(DEVICE)
for epoch in range(50):
    running_loss = 0.0
    for step, (b_x,b_y)in enumerate(trainloader):
        outputs = mynet(b_x.to(DEVICE)) # 喂给 net 训练数据 x, 输出预测值
        loss = loss_func(outputs, b_y.to(DEVICE)) # 计算两者的误差
        optimizer.zero_grad() # 清空上一步的残余更新参数值
        loss.backward() # 误差反向传播, 计算参数更新值
        optimizer.step() # 将参数更新值施加到 net 的 parameters 上
        writer.add_scalar("loss",loss.item(),step)
        # 打印状态信息
        running_loss += loss.item()
        #if(lost>loss.item()):
        #    lost = loss.item()
        #    torch.save(mynet.cpu(),str(loss.item()*100)+"ciarf.pth")
        #    mynet = mynet.train().to(DEVICE)

        #if step % 1000 == 999:    # 每2000个批次打印一次
        #    print('[%d, %5d] loss: %.3f' %
        #          (epoch + 1, step + 1, running_loss / 2000))
        #    running_loss = 0.0
    print('[%d] loss: %.3f' %
            (epoch + 1, running_loss / 1000))
    running_loss = 0.0
print('Finished Training')
writer.close()

RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED

In [ ]:
end = time.process_time()
print(end-start)

In [ ]:
#1483.984375

In [ ]:
mynet = mynet.eval().to(DEVICE)

In [ ]:
all_counter=0
correct_counter=0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)
    out = mynet(inputs)
    out = out.detach().cpu().argmax(1)
    t = labels.cpu()
    for m in range(len(t)):
        all_counter += 1
        if t[m] == out[m]:
            correct_counter += 1

print(correct_counter, all_counter, correct_counter / all_counter)

In [ ]:
#mynet=torch.load("0.7026215083897114ciarf.pth").to(DEVICE)
all_counter=0
correct_counter=0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)
    out = mynet(inputs)
    out = out.detach().cpu().argmax(1)
    t = labels.cpu()
    for m in range(len(t)):
        all_counter += 1
        if t[m] == out[m]:
            correct_counter += 1

print(correct_counter, all_counter, correct_counter / all_counter)